# Part 2: Creating your first Experiment in CRIPT 
---

In this notebook we will create document our first experiment, and all the associated data. The following example is the anionic polymerization of styrene with secBuLi in a mixture of THF and toluene. The reaction is then quenched with butanol and precipitated into methanol to obtain the final product, polystyrene.

<p style="text-align:center;"><img src="../Jupyter_notebook/supporting_files/pics/P2_polystyrene_synthesis.svg" alt="Polystyrene_synthesis" width="600" style="background-color:white;" />



---

We will start with the basics: importing 'CRIPT module', and connecting to the CRIPT database.

<u>Action:</u> Replace 'user = ###@###.##' with the email address you used for your `User node` or 'user uid'. Then run the cell. <span style="color:gray">(You can run a cell by either clicking on it and hitting the 'play/run' button in the toolbar or click the cell and hit "shift" + "enter".)</span>

<u>Output:</u> You should see "Import sucessful!", "Connection to database '####' successful.", "Login as "###" was sucessfully." 

In [1]:
import cript as C
print("Import successful!")

# Connect to database
db_username = "DW_cript"
db_password = "YXMaoE1"
db_project = "cript_testing"
db_database = "test"
user = "johndoe13@cript.edu"  # Put your email or uid here
db = C.CriptDB(db_username, db_password, db_project, db_database, user)

Import successful!
Connection to database 'test' successful.
Login as 'John Doe' was successful.



<span style="color:SteelBlue"> This previous block of code is the standard code you should type first for all future projects. </span> 

---

## Collection and Experiment Nodes

Before diving directly into this example we look at creating two organizaiton CRIPT nodes: `C.Collection()` and `C.Experiment()`.

<span style="color:SteelBlue">The `Experiment node` is the main organization node that will hold the data assoisated with an experiment. With that said, providing a formal defination of an experiment is challangeing.  </span> 

We will start by creating a `collection node()` in which we will associate with our "tutorial" group we create in part.


<u>Action:</u> Run the following cell.

<u>Output:</u> Print out of node. 

---

In [2]:
collection = C.Collection("Anionic Polymerizations")
print(collection)

{
  "class_": "Collection",
  "model_version": "0.0.1",
  "name": "Anionic Polymerizations"
}


Now that we create the collection we will want to save it and pick a group to own it. So lets view what groups we are apart of again. To do that, we can use the `db.view()` function again and pass it the generic CRIPT `Group node`. This will return just the groups you are apart of. Which at this point should just be two.

<u>Action:</u> Run the following cell.

<u>Output:</u> You should see two groups in the list. "tutorial_###" which is the node you created in part 1 and "CRIPT_community" the node you just joined in part 1.


In [3]:
groups = db.view(C.Group)


number  name                          uid                           
------------------------------------------------------------
0       tutorial_john                 610b220ee1756ffe3cc73495      
1       CRIPT_community               610a9f05ad3873e83363051e      



We will put this collection into "tutorial_####". To do this, we just need to save the node and pass it the group we want to save it to with it.

<span style="color:SteelBlue">In the background it will save the collection and update the group in the database. </span> 

<u>Action:</u> Change the number in the brackets to select the "tutorial_####" group. Then run the following cell.

<u>Output:</u> "Save of 'Anionic Polymerizations' was successful." and "Update of 'tutorial_john' successful!". We will also see the "uid" of the collection node.


In [4]:
db.save(collection, groups[0])

Save of 'Anionic Polymerizations' was successful.
Update of 'tutorial_john' successful!


'6112dcee356fd5289aa3d75c'

---

Now we have a collection made, let make our `Experiment node`.

<u>Action:</u> Run the following cell.

<u>Output:</u> Print out of the `Experiment node`.


In [5]:
expt = C.Experiment("Anionic Polymerization of Styrene with SecBuLi")
print(expt)

{
  "class_": "Experiment",
  "model_version": "0.0.1",
  "name": "Anionic Polymerization of Styrene with SecBuLi"
}


To view our collections of our user node

In [6]:
coll = db.view(C.Collection)


number  key                           name                          uid                           
------------------------------------------------------------
0       .tutorial_john                Anionic Polymerizations       6112da4b27fafff76d24f314      
1       .tutorial_john                Anionic Polymerizations       6112dcee356fd5289aa3d75c      



In [7]:
db.save(expt, coll[0])

Save of 'Anionic Polymerization of Styrene with SecBuLi' was successful.


'6112dcf9356fd5289aa3d75d'

In [10]:
db.view(coll[0]["uid"])

{'_id': ObjectId('6112da4b27fafff76d24f314'), 'created_date': datetime.datetime(2021, 8, 10, 19, 58, 3, 192000), 'name': 'Anionic Polymerizations', 'class_': 'Collection', 'model_version': '0.0.1', 'last_modified_date': datetime.datetime(2021, 8, 10, 19, 58, 3, 192000)}


{'_id': ObjectId('6112da4b27fafff76d24f314'),
 'created_date': datetime.datetime(2021, 8, 10, 19, 58, 3, 192000),
 'name': 'Anionic Polymerizations',
 'class_': 'Collection',
 'model_version': '0.0.1',
 'last_modified_date': datetime.datetime(2021, 8, 10, 19, 58, 3, 192000)}

---
---

## Materal Node

### Creating a Material Node


In [ ]:
help(cript.Material.__init__)

we see that the only required paramter is 'identifier'. Looking at the :param we see it is c_identifier meaning it is a CRIPT object.

In [ ]:
help(cript.Identifiers.__init__)

The minimuim requirement is the preferred_name, but its best to put as much as possible.

In [ ]:
iden = [cript.Identifiers(
    preferred_name="styrene",
    names=["vinylbenzene", "phenylethylene", "ethenylbenzene"],
    chem_formula="C8H8",
    smiles="C=Cc1ccccc1",
    cas="100-42-5",
    pubchem_cid="7501",
    inchi_key="PPBRXRYQALVLMV-UHFFFAOYSA-N"
)]

print(iden[0])

In [ ]:
isinstance(iden[0], tuple(C.cript_types.values()))

In [ ]:
mat_styrene = cript.Material(
    identifier=iden,
    keywords=["styrene"],
    storage=[
        cript.Cond(key="temp", value=-20, unit=Unit("degC")),
        cript.Cond(key="atm", value="argon")
    ]
    )

In [ ]:
 unit=Unit("degC")

In [ ]:
help(unit)

In [ ]:
# add properties
prop = [cript.Prop(mat_id=0, key="phase", value="liquid"),
        cript.Prop(mat_id=0, key="color", value="colorless"),
        cript.Prop(mat_id=0, key="mw", value=104.15, unit=Unit("g/mol"), method="prescribed"),
        cript.Prop(mat_id=0, key="density", value=0.906, unit=Unit("g/ml"),
                   conditions=[cript.Cond(key="temp", value=25, unit=Unit("degC"))]
                   ),
        cript.Prop(mat_id=0, key="bp", value=145, unit=Unit("degC"),
                   conditions=[cript.Cond(key="pressure", value=1, unit=Unit("atm"))]
                   ),
        cript.Prop(mat_id=0, key="mp", value=-30, unit=Unit("degC"),
                   conditions=[cript.Cond(key="pressure", value=1, unit=Unit("bar"))]
                   ),
        cript.Prop(mat_id=0, key="solubility", value=0.3, unit=Unit("g/mol"),
                   conditions=[
                             cript.Cond(key="temp", value=25, unit=Unit("degC")),
                             cript.Cond(key="solvent", value="water", unit=Unit("bar"))
                         ]
                   ),
        ]

node.properties = prop

In [ ]:
saves both

In [ ]:
db.save(mat_styrene, expt)


---

### Adding a Existing Material Node


In [ ]:
query = {
    "field": "identifier.name",
    "type" : "$regex",
    "value": "toluene"
}
tol_query = db.view(cript.Material, query)
query = {
    "field": "identifier.name",
    "type" : "$regex",
    "value": "thf"
}
thf_query = db.view(cript.Material, query)

In [ ]:
tol_query[0]

In [ ]:
expt.c_material = [tol_query[0], thf_query[0]]

---

### Copying an existing Material Node

In [ ]:
query = {
    "field": "identifier.name",
    "type" : "$regex",
    "value": "BuLi"
}
BuLi_query = db.view(cript.Material, query)

In [ ]:
BuLi_query[0]

In [ ]:
BuLi_node = cript.copy(BuLi_query[0])
BuLi_node.properties["conc"] = "1.2"

In [ ]:
db.save(BuLi_node, expt)

In [ ]:
globals()

In [ ]:
import cript

In [ ]:
db.update(expt)


---
---

## Key Takeaways

* You have 

---
---

### What's next?
